In [27]:
import torch
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import yaml

In [11]:
metadata_df = pd.read_csv('/global/D1/homes/vajira/data/sperm/visem/visem-dataset/videos/data/metadata.csv')

In [12]:
metadata_df.head()

,file_name,Serum C14:0 (myristic acid),Serum C16:0 (palmitic acid),Serum C16:1 (palmitoleic acid),Serum C18:0 (stearic acid),Serum C18:1 n-9 (oleic acid),Serum total C18:1,"Serum C18:2 n-6 (linoleic acid, LA)","Serum C18:3 n-6 (gamma-linoleic acid, GLA)",Serum C20:1 n-9,...,"High DNA stainability, HDS (%)","DNA fragmentation index, DFI (%)",Seminal plasma anti-Müllerian hormone (AMH) (pmol/L),Serum total testosterone (nmol/L),Serum oestradiol (nmol/L),"Serum sex hormone-binding globulin, SHBG (nmol/L)","Serum follicle-stimulating hormone, FSH (IU/L)","Serum Luteinizing hormone, LH (IU/L)",Serum inhibin B (ng/L),"Serum anti-Müllerian hormone, AMH (pmol/L)"
0,1.avi,"0,36","29,72","0,64","13,67","9,00","10,90","19,48","0,30","0,16",...,13,29,64,"16,8","0,13",15,"2,3","4,2",289,54
1,2.avi,"0,28","31,22","0,47","11,84","9,02","11,29","20,34","0,22","0,25",...,13,57,423,15,"0,15",43,"7,2","2,7",164,19
2,3.avi,"0,36","27,95","0,47","16,57","8,65","10,36","23,75","0,26","0,19",...,11,12,37,"8,2","0,18",32,"4,2","4,8",67,47
3,4.avi,"0,32","28,10","0,64","16,09","8,56","10,13","21,52","0,17","0,11",...,10,15,257,"11,2","0,15",11,"8,5","5,6",132,29
4,5.avi,"0,40","29,94","0,80","14,17","9,47","11,02","22,29","0,38","0,15",...,21,13,2203,"11,9","0,14",18,"2,6","2,1",139,68


In [14]:
#metadata_df['Sperm concentration (x10⁶/mL)']

In [15]:
# Plot sperm concentration distribution
#metadata_df.hist(bins=3, xrot=90)

In [20]:
#print(list(metadata_df.columns))

In [19]:
#skf = StratifiedKFold(n_splits=5)

In [18]:
#folds = skf.split(metadata_df['file_name'], metadata_df['Sperm concentration (x10⁶/mL)'])

In [21]:
#folds

In [17]:
#print(list(metadata_df['Sperm concentration (x10⁶/mL)']))

In [22]:
metadata_df['Sperm concentration (x10⁶/mL)'] = metadata_df['Sperm concentration (x10⁶/mL)'].str.replace(",", ".").astype('float')

In [16]:
#list(metadata_df.index)

In [23]:
out = pd.qcut(metadata_df['Sperm concentration (x10⁶/mL)'], q=5, labels=["lower", "low", "medium", "high", "higher"])

In [24]:
out

0       high
1        low
2        low
3        low
4        low
       ...  
80    medium
81     lower
82    higher
83       low
84    higher
Name: Sperm concentration (x10⁶/mL), Length: 85, dtype: category
Categories (5, object): ['lower' < 'low' < 'medium' < 'high' < 'higher']

In [25]:
# combine out with metadata_df using index
metadata_df['folds'] = out

In [80]:
folds

<generator object _BaseKFold.split at 0x1553c7108890>

In [43]:
skf = StratifiedKFold(n_splits=5)
folds = skf.split(metadata_df['file_name'], metadata_df["folds"])

yaml_data = {}
metadata_file = "data/metadata.csv"
yaml_data['config'] = metadata_file
mlist = []


for i, (train_i, test_i)in enumerate(folds):
    print(i)
    print(train_i)
    print(test_i) # test_i is the index of the test set or fold
    file_names = metadata_df['file_name'].iloc[test_i]
    print(file_names)
    # select rows from metadata_df using train_i and test_i
    #train_df = metadata_df.iloc[train_i]
    test_df = metadata_df.iloc[test_i]
    print("count of folds", test_df['folds'].value_counts())

    # Add data to yaml file
    mlist.append({"split": f"fold_{i}", 
                 "path": ["data/" + p for p in file_names.to_list() + [metadata_file]]
                    })

yaml_data["data_files"] = mlist


# Save the yaml file
with open('folds.yaml', 'w') as file:
    documents = yaml.dump(yaml_data, file)



    # use the following formart to save the output of test_i into a yaml file
    # formart: 
  #- split: fold_01
  #  path:
  #  - data/1.avi
  #  - data/2.avi
  #  - data/metadata.csv
  #- split: fold_02
  #  path:
  #  - data/3.avi
  #  - data/4.avi
  #  - data/metadata.csv

    # save the output of test_i into a yaml file


0
[ 4  9 11 14 18 20 21 22 24 25 27 28 29 30 31 32 33 34 35 36 37 38 39 40
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84]
[ 0  1  2  3  5  6  7  8 10 12 13 15 16 17 19 23 26]
0      1.avi
1      2.avi
2      3.avi
3      4.avi
5      6.avi
6      7.avi
7      8.avi
8      9.avi
10    11.avi
12    13.avi
13    14.avi
15    16.avi
16    17.avi
17    18.avi
19    20.avi
23    24.avi
26    27.avi
Name: file_name, dtype: object
count of folds folds
medium    4
high      4
lower     3
low       3
higher    3
Name: count, dtype: int64
1
[ 0  1  2  3  5  6  7  8 10 12 13 15 16 17 19 22 23 26 27 28 29 34 37 39
 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64
 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84]
[ 4  9 11 14 18 20 21 24 25 30 31 32 33 35 36 38 40]
4      5.avi
9     10.avi
11    12.avi
14    15.avi
18    19.avi
20    21.avi
21    22.avi
24    25.avi
25    26.av

In [54]:
out.value_counts()

Sperm concentration (x10⁶/mL)
lower     17
low       17
medium    17
high      17
higher    17
Name: count, dtype: int64

In [24]:
for i, (_, _)in enumerate(folds):
    print(i)
    #print(file_name)
    #print(sperm_concentration)
    #print(metadata_df['Sperm concentration (x10⁶/mL)'][file_name])
    #print(metadata_df['Sperm concentration (x10⁶/mL)'][sperm_concentration])
    #print('---')

    

ValueError: n_splits=5 cannot be greater than the number of members in each class.